In [2]:
import pandas as pd
import pyarrow.parquet as pq

# Uploading the two df for metadata

In [3]:

df_im = pq.read_table("../sourcedata/clips_metadata_from_im.parquet")
df_im = df_im.to_pandas(types_mapper=None)
df_base = pq.read_table("../sourcedata/clips_metadata_with_patterns.parquet")
df_base = df_base.to_pandas(types_mapper=None)
df_im = df_im.rename(columns={"StateClipCode": "ClipCode"})

df_base['Cleared'] = df_base['Cleared'].map({'True': 1, 'False': 0})
df = pd.concat([df_base, df_im], ignore_index=True)
df['Scene'] = df['Scene'].astype(int)
df['Average_speed'] = df['Average_speed'].astype(float)
df['Duration'] = df['Duration'].astype(float)
df['X_Traveled'] = df['X_Traveled'].astype(float)
df

,Model,StateFileName,LevelFullName,Scene,ClipCode,TotalFrames,Bk2Filepath,GameName,Subject,World,...,Pattern_Enemy stair valley,Pattern_Gap stair valley,Pattern_Reward,Pattern_Moving platform,Pattern_Flagpole,Pattern_Beginning,Pattern_Bonus zone,Pattern_Waterworld,Learning_Phase,json_path
0,human,None,w1l3,0,103100000111,2628,None,SuperMarioBros-Nes,sub-01,1,...,0,0,0,0,0,1,0,0,Late discovery,NaN
1,human,None,w1l1,10,101000003476,4831,None,SuperMarioBros-Nes,sub-01,1,...,0,0,0,0,0,0,0,0,Early discovery,NaN
2,human,None,w1l3,6,103010001275,2528,None,SuperMarioBros-Nes,sub-01,1,...,0,0,0,1,0,0,0,0,Early discovery,NaN
3,human,None,w2l1,13,104000002344,3354,None,SuperMarioBros-Nes,sub-01,2,...,0,0,0,0,0,0,0,0,Early discovery,NaN
4,human,None,w1l2,6,102010002114,3090,None,SuperMarioBros-Nes,sub-01,1,...,0,0,0,0,0,0,0,0,Early discovery,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
617546,sub-06_epoch=0-step=4000,scene_clips/sub-06/ses-008/beh/savestates/sub-...,w4l3,2,00803030001282,165,sub-06_epoch=0-step=4000/sub-06/ses-008/beh/bk...,SuperMarioBros-Nes,06,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,../../../github/mario_curiosity.scene_agents/o...
617547,sub-06_epoch=0-step=4000,scene_clips/sub-06/ses-008/beh/savestates/sub-...,w1l3,8,00803020004180,154,sub-06_epoch=0-step=4000/sub-06/ses-008/beh/bk...,SuperMarioBros-Nes,06,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,../../../github/mario_curiosity.scene_agents/o...
617548,sub-06_epoch=0-step=4000,scene_clips/sub-06/ses-008/beh/savestates/sub-...,w5l3,3,00802010001104,477,sub-06_epoch=0-step=4000/sub-06/ses-008/beh/bk...,SuperMarioBros-Nes,06,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,../../../github/mario_curiosity.scene_agents/o...
617549,sub-06_epoch=0-step=4000,scene_clips/sub-06/ses-008/beh/savestates/sub-...,w6l3,6,00801010002778,4743,sub-06_epoch=0-step=4000/sub-06/ses-008/beh/bk...,SuperMarioBros-Nes,06,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,../../../github/mario_curiosity.scene_agents/o...


In [4]:
mask_im = df['Model'].str.startswith('sub-0')
df.loc[mask_im, 'Learning_Phase'] = df.loc[mask_im, 'Model'].str[:]
df.loc[mask_im, 'Subject'] = "im_"+df.loc[mask_im, 'Model'].str[:6]

mask_ppo = df['Model'].str.startswith('ep')
df.loc[mask_ppo, 'Learning_Phase'] = df.loc[mask_ppo, 'Model'].str[:]
df.loc[mask_ppo, 'Subject'] = "ppo"
df.loc[mask_ppo, 'Average_speed'] = df.loc[mask_ppo, 'X_Traveled'] / df.loc[mask_ppo, 'Duration']

print(df['Learning_Phase'].unique())
print(df['Subject'].unique())

['Late discovery' 'Early discovery' 'Early practice' 'Late practice'
 'ep-20' 'ep-2000' 'ep-4000' 'ep-6000' 'ep-8000'
 'sub-06_epoch=0-step=2000' 'sub-03_epoch=1-step=14908'
 'sub-01_epoch=0-step=6500' 'sub-03_epoch=0-step=4000'
 'sub-01_epoch=0-step=2000' 'sub-02_epoch=0-step=8000'
 'sub-03_epoch=0-step=500' 'sub-05_epoch=0-step=4000'
 'sub-06_epoch=0-step=5500' 'sub-05_epoch=0-step=1500'
 'sub-05_epoch=0-step=3000' 'sub-03_epoch=1-step=11408'
 'sub-05_epoch=0-step=500' 'sub-01_epoch=0-step=500'
 'sub-02_epoch=0-step=3000' 'sub-06_epoch=0-step=500'
 'sub-03_epoch=0-step=7500' 'sub-02_epoch=0-step=5500'
 'sub-05_epoch=0-step=5000' 'sub-06_epoch=0-step=7000'
 'sub-02_epoch=0-step=500' 'sub-01_epoch=0-step=3500'
 'sub-02_epoch=0-step=10000' 'sub-01_epoch=0-step=5000'
 'sub-06_epoch=0-step=4000']
['sub-01' 'sub-02' 'sub-03' 'sub-05' 'sub-06' 'ppo' 'im_sub-06'
 'im_sub-03' 'im_sub-01' 'im_sub-02' 'im_sub-05']


# Uploading the df for variables

In [5]:
tab_variable = pq.read_table("../sourcedata/df_variables.parquet")
df_variables = tab_variable.to_pandas()
df_variables['player_x_pos'] = df_variables['player_x_posHi']*255 + df_variables['player_x_posLo']
df_variables['Scene'] = df_variables['Scene'].astype(int)

In [13]:
df_variables

,Subject,World,Level,Scene,LevelFullName,SceneFullName,ClipCode,Learning_Phase,player_x_posHi,player_x_posLo,player_y_pos,json_path,player_x_pos
0,ppo,6,1,5,w6l1,6-1-5,00403040000625,None,"[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","[220, 222, 225, 227, 230, 232, 235, 237, 240, ...","[90, 94, 98, 102, 106, 110, 114, 118, 122, 126...",/home/hugo/github/mario.scenes/outputdata/outp...,"[1240, 1242, 1245, 1247, 1250, 1252, 1255, 125..."
1,ppo,6,1,2,w6l1,6-1-2,00403030000260,None,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[141, 143, 146, 148, 151, 153, 156, 158, 161, ...","[148, 144, 140, 136, 133, 129, 126, 123, 120, ...",/home/hugo/github/mario.scenes/outputdata/outp...,"[396, 398, 401, 403, 406, 408, 411, 413, 416, ..."
2,ppo,5,3,5,w5l3,5-3-5,00402060001709,None,"[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","[40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 4...","[176, 176, 176, 176, 176, 176, 176, 176, 176, ...",/home/hugo/github/mario.scenes/outputdata/outp...,"[1060, 1060, 1060, 1060, 1060, 1060, 1060, 106..."
3,ppo,6,2,3,w6l2,6-2-3,00404050002452,None,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[56, 58, 61, 63, 66, 68, 71, 73, 76, 78, 81, 8...","[73, 77, 81, 85, 89, 93, 97, 101, 105, 109, 11...",/home/hugo/github/mario.scenes/outputdata/outp...,"[566, 568, 571, 573, 576, 578, 581, 583, 586, ..."
4,ppo,5,2,10,w5l2,5-2-10,00401080003246,None,"[7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, ...","[56, 57, 59, 60, 61, 62, 63, 64, 65, 66, 67, 6...","[142, 144, 144, 144, 144, 144, 144, 144, 144, ...",/home/hugo/github/mario.scenes/outputdata/outp...,"[1841, 1842, 1844, 1845, 1846, 1847, 1848, 184..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
336841,ppo,3,2,3,w3l2,3-2-3,01501020000527,ep-8000,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[61, 62, 64, 66, 68, 69, 71, 73, 75, 77, 79, 8...","[176, 176, 176, 176, 176, 176, 176, 176, 176, ...",/home/hugo/Téléchargements/mario_learning-sour...,"[571, 572, 574, 576, 578, 579, 581, 583, 585, ..."
336842,ppo,6,1,4,w6l1,6-1-4,01503040001236,ep-8000,"[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ...","[203, 205, 206, 208, 209, 211, 212, 214, 215, ...","[172, 168, 164, 160, 157, 153, 150, 147, 144, ...",/home/hugo/Téléchargements/mario_learning-sour...,"[968, 970, 971, 973, 974, 976, 977, 979, 980, ..."
336843,ppo,7,3,3,w7l3,7-3-3,01501000002470,ep-8000,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[172, 174, 177, 179, 181, 183, 186, 188, 190, ...","[123, 118, 113, 108, 104, 100, 96, 92, 88, 85,...",/home/hugo/Téléchargements/mario_learning-sour...,"[682, 684, 687, 689, 691, 693, 696, 698, 700, ..."
336844,ppo,5,2,1,w5l2,5-2-1,01502010000251,ep-8000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[184, 186, 188, 191, 193, 195, 197, 199, 201, ...","[104, 103, 102, 103, 103, 105, 107, 109, 112, ...",/home/hugo/Téléchargements/mario_learning-sour...,"[184, 186, 188, 191, 193, 195, 197, 199, 201, ..."


# Dashboard

In [12]:
import os
import glob
import numpy as np
import pandas as pd
from PIL import Image

import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.colors import sample_colorscale

import ipywidgets as widgets
from IPython.display import display, clear_output
from pathlib import Path

# -------------------------- #
# ---------- DATA ---------- #
# -------------------------- #

metrics = ["Average_speed"]
means = df[metrics].mean(numeric_only=True)
stds  = df[metrics].std(numeric_only=True).replace(0, np.nan)
for m in metrics:
    zc = f"{m}_z"
    df[zc] = (df[m] - means[m]) / stds[m]

## top-right

order_hum = ['sub-01', 'sub-02', 'sub-03', 'sub-05', 'sub-06']
order_2phases = ["discovery", "practice"]
colors_2phases = dict(zip(order_2phases, sample_colorscale("Viridis", [0.33,0.66])))
position_2phase = dict(discovery = "bottom left", practice="bottom right")
hum_2phases = dict(zip(order_hum, [order_2phases]*len(order_hum)))

## bottom

order_ppos = ['im_sub-01', 'im_sub-02', 'im_sub-03', 'im_sub-05', 'im_sub-06', 'ppo']
order_4phases = ["Early discovery", "Late discovery", "Early practice", "Late practice"]
ppos_phases = [['sub-01_epoch=0-step=500', 'sub-01_epoch=0-step=2000', 'sub-01_epoch=0-step=3500', 'sub-01_epoch=0-step=5000', 'sub-01_epoch=0-step=6500'],
                ['sub-02_epoch=0-step=500', 'sub-02_epoch=0-step=3000', 'sub-02_epoch=0-step=5500', 'sub-02_epoch=0-step=8000', 'sub-02_epoch=0-step=10000'],
                ['sub-03_epoch=0-step=500', 'sub-03_epoch=0-step=4000', 'sub-03_epoch=0-step=7500', 'sub-03_epoch=1-step=11408', 'sub-03_epoch=1-step=14908'],
                ['sub-05_epoch=0-step=500', 'sub-05_epoch=0-step=1500', 'sub-05_epoch=0-step=3000', 'sub-05_epoch=0-step=4000', 'sub-05_epoch=0-step=5000'],
                ['sub-06_epoch=0-step=500', 'sub-06_epoch=0-step=2000', 'sub-06_epoch=0-step=4000', 'sub-06_epoch=0-step=5500', 'sub-06_epoch=0-step=7000'],
                ['ep-20', 'ep-2000', 'ep-4000', 'ep-6000', 'ep-8000']]
phases_everyone = dict(zip(order_hum+order_ppos, [order_4phases]*len(order_hum) + ppos_phases))
order_all_phases = order_4phases + [phase for subset in ppos_phases for phase in subset]
color_4phases= sample_colorscale("Viridis", [0.00,0.33, 0.66, 1.00])
color_ckpt = sample_colorscale("magma", [0.2,0.4,0.6,0.8,1.0])
colors_phase_ckpt = dict(zip(order_all_phases, color_4phases+color_ckpt*6))

# ----------------------------- #
# ---------- HELPERS ---------- #
# ----------------------------- #

def scenes_for_level(df, level):
    return sorted(df.loc[df["LevelFullName"] == level, "Scene"].dropna().unique().tolist())

def subjects_for(df, level, scene):
    m = (df["LevelFullName"] == level) & (df["Scene"] == scene)
    return sorted(df.loc[m, "Subject"].dropna().unique().tolist())

def get_subset(df, level, scene):
       m = (
        (df["LevelFullName"] == level) &
        (df["Scene"] == scene))
       return df.loc[m].copy()

def load_scene_image(level, scene):
    scene_fullname = f"{level}s{scene}"
    root = Path.cwd().parent
    path = os.path.join(root, 'sourcedata', 'mario_backgrounds', 'scene_backgrounds', f"{scene_fullname}.png")
    if os.path.exists(path):
        im = Image.open(path).convert("RGB")
        return np.array(im), path
    return None, path

def compute_delta_tot(df, phase_to_ckpt, metric):

    df_tmp = df.copy()
    df_tmp["ckpt"] = df_tmp["Learning_Phase"].map(phase_to_ckpt)
    df_ckpt =df_tmp.groupby(["Subject", "ckpt"])[metric].mean().reset_index(name=metric)

    idx_max = df_ckpt.groupby("Subject")["ckpt"].idxmax()
    df_max = df_ckpt.loc[idx_max, ["Subject", metric]].rename(columns={metric: metric+"_max"})    
    idx_min = df_ckpt.groupby("Subject")["ckpt"].idxmin()
    df_min = df_ckpt.loc[idx_min, ["Subject", metric]].rename(columns={metric: metric+"_min"})

    results = df_min.merge(df_max, on="Subject")
    results['delta_tot'] = results[metric+"_max"] - results[metric+"_min"]
    return results


def compute_deltas(df, phase_to_ckpt, metric):

    df_tmp = df.copy()
    df_tmp["ckpt"] = df_tmp["Learning_Phase"].map(phase_to_ckpt)

    df_ckpt = (
        df_tmp.groupby(["Subject", "ckpt"])[metric]
        .mean()
        .reset_index(name=metric)
    )

    df_wide = df_ckpt.pivot(index="Subject", columns="ckpt", values=metric)
    df_shifted = df_wide.shift(-1, axis=1)
    df_delta = df_shifted - df_wide
    df_delta_shifted = df_delta.shift(1, axis=1)

    return df_delta_shifted

def get_traces(level, scene):
    root = Path.cwd().parent
    paths_traces = glob.glob(os.path.join(root, 'sourcedata', 'traces', 'sub-*','scenes', f'*scene-{level}s{scene}_traces.png'))
    paths_traces = sorted(paths_traces)
    imgs_list = []
    for path in paths_traces:
        img = Image.open(path).convert("RGB")
        imgs_list.append(np.array(img))
    return imgs_list, paths_traces

def concat_traces(img_scene, imgs_traces):
    img_tot = img_scene
    for img in imgs_traces:
        idx_marge = np.where(img.mean(axis=2)==255)
        slide_img = (slice(idx_marge[0].min(), idx_marge[0].max()+1), slice(idx_marge[1].max()+1, img.shape[1]))
        img = img[slide_img]
        img_tot = np.concat([img_tot, np.zeros([img.shape[0], 5, 3]),img], axis=1)

    return img_tot

def mad(values):
    values = np.array(values)
    return np.median(np.abs(values - np.median(values)))

def get_mads(df):

    df_exp = (
        df
        .explode(['player_x_pos', 'player_y_pos'])
    )

    df_mad = (
        df_exp
        .groupby(['Subject', 'Learning_Phase', 'player_x_pos'])['player_y_pos']
        .apply(lambda g: mad(g.tolist()))
        .reset_index(name='MAD_y_for_x')
    )

    df_mad_mean = (
        df_mad
        .groupby(['Subject', 'Learning_Phase'])['MAD_y_for_x']
        .mean()
        .reset_index(name='MAD_mean')
    )

    return df_mad_mean

# ----------------------------- #
# ------------ MAIN ----------- #
# ----------------------------- #

def make_dashboard(df, level, scene):
    df_wls = get_subset(df, level, scene)
    df_var_wls = get_subset(df_variables, level, scene)
    var_int = ['player_x_pos', 'player_y_pos', 'Learning_Phase', 'Subject']
    df_var_wls = df_var_wls[var_int]

    df_human = df_wls[df_wls["Model"] == "human"].copy()
    df_ppos = df_wls[df_wls["Model"] != "human"].copy()

    fig = make_subplots(
        rows=3, cols=2,
        specs=[[{"type": "xy", 'colspan': 2}, None], 
               [{"type": "xy"}, {"type": "xy"}], 
               [{"type": "xy"}, {"type": "xy"}]],
        column_widths=[0.5, 0.5],
        row_heights=[0.3, 0.35, 0.35],
        horizontal_spacing=0.1,
        subplot_titles=("Scene Background with Players Traces","mean MADs per Learning Phase ",  " Num. Clips per Learning Phase", "Clearance Through Learning-Phases", " Z Speed Through Learning-Phases")
    )
    fig.update_layout(height=6000)
    ########################################
    # Top-TOP: non-interactive background #
    ########################################

    img_arr, img_path = load_scene_image(level, scene)
    traces_arr, traces_path = get_traces(level, int(scene))
    img_final = concat_traces(img_arr, traces_arr)

    if img_arr is not None:
        fig.add_trace(go.Image(z=img_final, hoverinfo="skip"), row=1, col=1)
    else:
        fig.add_annotation(row=1, col=1, x=0.5, y=0.5, xref="x1", yref="y1",
                           text=f"No image found:<br>{img_path}", showarrow=False)
        
    x_labels = ('empty', 'sub_01', 'sub_02', 'sub_03', 'sub_05', 'sub_06')

    width = img_final.shape[1]
    n_imgs = len(x_labels)

    # Largeur d’une image individuelle (si elles ont toutes la même taille)
    w_single = width / n_imgs

    # Positions centrées
    tick_positions = np.arange(w_single/2, width, w_single)

    fig.update_xaxes(
        tickvals=tick_positions,
        ticktext=x_labels,
        title_text="Subjects",
        fixedrange=True,
        visible=True,
        row=1, col=1    
    )

    fig.update_yaxes(visible=False, fixedrange=True, row=1, col=1)

    #********************#
    # Mid-Left: Varience #
    #********************# 

    phase_to_human = {p: i % 4 for i, p in enumerate(order_4phases)}
    phase_to_checkpoint = {p: i % 5 for i, p in enumerate([phase for subset in ppos_phases for phase in subset])}
    phase_to_all = {**phase_to_human, **phase_to_checkpoint}
    mads = get_mads(df_var_wls)

    for sub in phases_everyone.keys():
        print(sub)
        for idx, i in enumerate(phases_everyone[sub]):
            df_phase = mads[(mads["Learning_Phase"] == i) & (mads["Subject"] == sub)]
            print(df_phase)
            if df_phase.empty:
                df_phase = pd.DataFrame({
                    "Subject": [sub],
                    "Learning_Phase": [i],
                    "MAD_mean": [0.0]
                })
            fig.add_trace(
                go.Bar(
                    x=df_phase["Subject"],
                    y=df_phase["MAD_mean"],
                    name=i,
                    marker_color=colors_phase_ckpt[i],
                    offsetgroup=f"chk{phase_to_all[i]}",
                    text=df_phase["MAD_mean"].round(2).astype(str),
                    textposition="inside",
                    cliponaxis=False,
                    hovertemplate="Value: %{y:.2f}<extra></extra>",
                    showlegend=False
                ),
                row=2, col=1
            )   
    fig.update_xaxes(visible=False, fixedrange=True, row=2, col=1)


    #*********************************************#
    # Mid-Right: bar (counts) with outside labels #
    #*********************************************#    

    df_count_tries = (
        df_human.groupby(["Subject", "Phase"])["ClipCode"]
        .count()
        .reset_index(name="count")
    )
    df_counts_cleared = (
        df_human.groupby(["Subject", "Phase"])["Cleared"]
        .mean()
        .reset_index(name="clearance")
    )
    show_legend = False
    legende_is_first_time = True
    for sub in hum_2phases.keys():
        sub_phases = df_count_tries[df_count_tries["Subject"] == sub]["Phase"].unique()
        for phase in hum_2phases[sub]:
            df_phase = df_count_tries[(df_count_tries["Phase"] == phase) & (df_count_tries["Subject"] == sub)]
            df_phase = df_phase.merge(
                df_counts_cleared[df_counts_cleared["Phase"] == phase],
                on=["Subject", "Phase"], how="left"
            )

            if df_phase.empty:
                df_phase = pd.DataFrame({
                    "Subject": [sub],
                    "Phase": [phase],
                    "count": [0],
                    "clearance": [0.0]
                })

            if set(order_2phases).issubset(set(sub_phases)):
                if legende_is_first_time:
                    show_legend = True
                    legende_is_first_time = False

            fig.add_trace(
                go.Bar(
                    x=df_phase["Subject"],
                    y=df_phase["count"],
                    name=phase,
                    marker_color=colors_2phases[phase],
                    textfont=dict(
                        size=13,
                        color='rgb(68, 1, 84)',       
                        family="Arial Black"
                    ),
                    text=df_phase["count"],
                    textposition="outside",
                    offsetgroup=phase,
                    cliponaxis=False,
                    hovertemplate="Phase: %{name}<br>Value: %{y}<extra></extra>",
                    showlegend=show_legend
                    ),
                row=2, col=2
            )

            fig.add_trace(
            go.Scatter(
                x=df_phase["Subject"],
                y=df_phase["clearance"] * df_phase["count"],
                mode="markers+text",
                offsetgroup=phase,
                showlegend=show_legend,
                text=(df_phase["clearance"] * 100).round(0).astype(str) + "%",
                textposition=position_2phase[phase],
                textfont=dict(
                    size=13,
                    color='rgb(68, 1, 84)',       
                    family="Arial Black"
                    ),
                marker=dict(
                    size=10,
                    color=colors_2phases[phase],
                    symbol="diamond-wide",
                    line=dict(width=1, color='rgb(68, 1, 84)')
                    ),
        #marker_symbol='line-ew',
                name=f"Success Rate at {phase} Phase ",
                ),
                row=2, col=2
            )
        show_legend = False

    fig.update_layout(barmode="group")
    fig.update_yaxes(title_text="N tries", row=2, col=2, range=[-3, df_count_tries["count"].max() * 1.2])
    fig.update_xaxes(title_text="Learning Phase", row=3, col=1)

    ###############################
    # Bottom Left: bar (clerance) #
    ###############################
    df_clearance = df_wls.groupby(["Subject", "Learning_Phase"])["Cleared"].mean().reset_index(name="Cleared")    
    
    df_deltas = compute_deltas(df_clearance, phase_to_all, "Cleared")

    for sub in phases_everyone.keys():
        for idx, i in enumerate(phases_everyone[sub]):
            df_phase = df_clearance[(df_clearance["Learning_Phase"] == i) & (df_clearance["Subject"] == sub)]
            if df_phase.empty:
                df_phase = pd.DataFrame({
                    "Subject": [sub],
                    "Learning_Phase": [i],
                    "Cleared": [0.0]
                })
                df_deltas.loc[sub, idx] = 0
            chk = phase_to_all[i]   # <-- NEW: group phases into 5 groups
            fig.add_trace(
                go.Bar(
                    x=df_phase["Subject"],
                    y=(df_phase["Cleared"]*100).round(0),
                    name=i,
                    marker_color=colors_phase_ckpt[i],
                    offsetgroup=f"chk{chk}",
                    text=(df_phase["Cleared"]*100).round(0).astype(str) + "%",
                    textposition="inside",
                    cliponaxis=False,
                    customdata=np.array((df_deltas.loc[sub, idx]).astype(float).round(2).astype(str) + " d").reshape(-1, 1),
                    hovertemplate="Value: %{y:.2f}<br>Delta with n-1: %{customdata[0]}<extra></extra>",
                    showlegend=False
                ),
                row=3, col=1
            )
    
    fig.update_yaxes(title_text="Clearance Rate", row=3, col=1, range=[-10, 120])

    simple_names = order_4phases + ["Checkpoint 1", "Checkpoint 2", "Checkpoint 3", "Checkpoint 4", "Checkpoint 5"]

    #creat a simple legend
    for label, color in zip(simple_names, color_4phases+color_ckpt):
        fig.add_trace(
            go.Scatter(
                x=[None], y=[None],      
                mode="markers",
                marker=dict(size=12, color=color),
                name=label,         
                showlegend=True,
            ),
            row=3, col=1
        )
    
    df_delta_tot = compute_delta_tot(df_clearance, phase_to_all, 'Cleared')


    for sub in df_delta_tot['Subject'].sort_values().unique():
        df_sub = df_delta_tot[df_delta_tot['Subject'] == sub]
        fig.add_trace(
            go.Scatter(
                x=df_sub['Subject'],
                y=[100],
                mode='text',
                text=(df_sub['delta_tot']).round(2).astype(str) + " d",
                textposition="top center",
                textfont=dict(
                size=10,
                color='rgb(68, 1, 84)',       
                family="Arial Black"
                ),
                name=f"Delta Total Clearance for {sub}",
                showlegend=False
            ),
            row=3, col=1
        )

    ###############################
    # Bottom Right: bar (speed) #
    ###############################

    df_speed = df_wls.groupby(["Subject", "Learning_Phase"])["Average_speed_z"].mean().reset_index(name="Average_speed_z")
    
    df_deltas = compute_deltas(df_speed, phase_to_all, "Average_speed_z")

    for sub in phases_everyone.keys():
        for idx, i in enumerate(phases_everyone[sub]):
            df_phase = df_speed[(df_speed["Learning_Phase"] == i) & (df_speed["Subject"] == sub)]
            if df_phase.empty:
                df_phase = pd.DataFrame({
                    "Subject": [sub],
                    "Learning_Phase": [i],
                    "Average_speed_z": [0.0]
                })
                df_deltas.loc[sub, idx] = 0
                
            chk = phase_to_all[i]
            fig.add_trace(
                go.Bar(
                    x=df_phase["Subject"],
                    y=df_phase["Average_speed_z"],
                    name=i,
                    marker_color=colors_phase_ckpt[i],
                    offsetgroup=f"chk{chk}",
                    text=df_phase["Average_speed_z"].round(0).astype(str),
                    textposition="inside",
                    cliponaxis=False,
                    customdata=np.array((df_deltas.loc[sub, idx]).astype(float).round(2).astype(str) + " d").reshape(-1, 1),
                    hovertemplate="Value: %{y:.2f}<br>Delta with n-1: %{customdata[0]}<extra></extra>",
                    showlegend=False
                ),
                row=3, col=2
            )
    max_y = max(df_speed["Average_speed_z"].max(), abs(df_speed["Average_speed_z"].min()))
    fig.update_yaxes(title_text="Average Speed Zed", row=3, col=2, range=[ -max_y*1.4, max_y.max()*1.4])

    simple_names = order_4phases+["Checkpoint 1", "Checkpoint 2", "Checkpoint 3", "Checkpoint 4", "Checkpoint 5"]

    # Ajouter une entrée de légende par couleur (invisible dans le graphe)
    for label, color in zip(simple_names, color_4phases+color_ckpt):
        fig.add_trace(
            go.Scatter(
                x=[None], y=[None],      
                mode="markers",
                marker=dict(size=12, color=color),
                name=label,
                showlegend=False,
            ),
            row=3, col=2
        )
    
    df_delta_tot = compute_delta_tot(df_speed, phase_to_all, "Average_speed_z")

    for sub in df_delta_tot['Subject'].sort_values().unique():
        df_sub = df_delta_tot[df_delta_tot['Subject'] == sub]
        fig.add_trace(
            go.Scatter(
                x=df_sub['Subject'],
                y=[df_speed["Average_speed_z"].max()],
                mode='text',
                text=(df_sub['delta_tot']).round(2).astype(str) + " d",
                textposition="top center",
                textfont=dict(
                size=10,
                color='rgb(68, 1, 84)',       
                family="Arial Black"
                ),
                name=f"Delta Total Clearance for {sub}",
                showlegend=False
            ),
            row=3, col=2
        )

    # Global layout tuned for clean initial render
    scene_fullname = f"{level}s{scene}"
    fig.update_layout(
        height=540,
        autosize=True,
        margin=dict(l=20, r=20, t=70, b=70),
        title_text=f"Level: {level} | Scene: {scene} ({scene_fullname})",
        legend=dict(orientation="h", y=-0.22, x=0.0),
        uniformtext_minsize=10, uniformtext_mode='hide',
        bargap=0.3,
    )
    # Shrink subplot titles, nudge slightly
    for ann in fig.layout.annotations:
        ann.font.size = 12
        ann.yshift = 6

    return fig

# ---------- WIDGETS ----------
levels = sorted(df["LevelFullName"].dropna().unique())

level_dd = widgets.Dropdown(
    options=levels, value=levels[0],
    description="Level:", layout=widgets.Layout(width="240px")
)

# Scene as SelectionSlider with exactly N positions (available scenes for the level)
def build_scene_slider(level):
    opts = scenes_for_level(df, level)
    if not opts:
        return widgets.SelectionSlider(options=[], value=None, description="Scene:", layout=widgets.Layout(width="500px"))
    # label each tick with the scene number; N positions = len(opts)
    return widgets.SelectionSlider(
        options=[(str(s), s) for s in opts],
        value=opts[0],
        description="Scene:",
        layout=widgets.Layout(width="500px"),
        continuous_update=False
    )

scene_slider = build_scene_slider(level_dd.value)

def current_subject_options():
    if scene_slider.value is None:
        return []
    return subjects_for(df, level_dd.value, scene_slider.value)

out = widgets.Output(layout=widgets.Layout(border="0px"))

# ---------- CALLBACKS ----------
def on_level_change(change):
    # rebuild scene slider with N positions for the selected level
    global scene_slider
    new_slider = build_scene_slider(level_dd.value)
    # swap the widget in place inside the UI
    controls.children = (level_dd, new_slider)
    # rebind handler
    new_slider.observe(on_scene_change, names="value")
    scene_slider.unobserve(on_scene_change, names="value")
    # update reference
    scene_slider = new_slider

    # update subject list
    subj_opts = current_subject_options()

    draw()

def on_scene_change(change):
    subj_opts = current_subject_options()
    draw()

def draw():
    with out:
        clear_output(wait=True)
        if (level_dd.value is None) or (scene_slider.value is None):
            print("No data available for the current selection.")
            return
        fig = make_dashboard(df, level_dd.value, scene_slider.value)
        # Use .show with minimal toolbar & no selection tools
        fig.show(config={"displaylogo": False, "modeBarButtonsToRemove": ["select", "lasso2d"]})

# Wire up observers
level_dd.observe(on_level_change, names="value")
scene_slider.observe(on_scene_change, names="value")

# ---------- DISPLAY FIRST, THEN DRAW (prevents initial overlap) ----------
controls = widgets.HBox([level_dd, scene_slider])
display(controls, out)
draw()  # clean initial render



Output(layout=Layout(border_bottom='0px', border_left='0px', border_right='0px', border_top='0px'))